In [ ]:
CapacityMetricsWorkspace = 'Microsoft Fabric Capacity Metrics'
CapacityMetricsDataset = 'Fabric Capacity Metrics'

### Get the workspace id

In [ ]:
workspaceId = spark.conf.get('trident.workspace.id')
print(f"{workspaceId=}")

In [ ]:
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}', headers=header)
workspaceName = response.json().get('displayName')
print(f'{workspaceName=}')

### Create lakehouses

In [ ]:
from notebookutils import mssparkutils

for lakehouseName in ['LH_SampleData', 'LH_QueryResults']:
    try:
        lakehouseItem = mssparkutils.lakehouse.create(lakehouseName, "", workspaceId)
    except Exception as e:
        # print(e)
        pass

LH_SampleData = mssparkutils.lakehouse.get("LH_SampleData", workspaceId)
LH_QueryResults = mssparkutils.lakehouse.get("LH_QueryResults", workspaceId)
print(f'{LH_SampleData=}')
print(f'{LH_QueryResults=}')

### Get notebook definition from github

In [ ]:
import requests, json, base64
url = "https://raw.githubusercontent.com/bretamyers/FabricTools/dev/FabricDWQueryCostAnalyzer/NB_DW_Load_Cost_Analyzer.ipynb"
response = requests.get(url)

b = base64.b64encode(bytes(json.dumps(response.json()), 'utf-8')) # convert to bytes
base64_str = b.decode('utf-8') # convert bytes to string
notebookDefDict = json.loads(base64.b64decode(base64_str).decode('utf-8')) # convert base64 bytes to string and then to dictionary
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse'] = LH_QueryResults.get('id')
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse_name'] = LH_QueryResults.get('displayName')
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse_workspace_id'] = LH_QueryResults.get('workspaceId')
notebookDefBase64 = base64.b64encode(json.dumps(notebookDefDict).encode('utf-8')).decode('utf-8') # convert dictionary back to base64 string

### Create notebook from the definition

In [ ]:
import requests, json, time, base64

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

body = {
    "displayName": "NB_DW_Load_Cost_Analyzer",
    "description": "A notebook to run data warehouse queries and capture the duration and cost of those queries.",
    "definition": {
        "format": "ipynb",
        "parts": [
            {
                "path": "notebook-content.py"
                ,"payload": notebookDefBase64
                ,"payloadType": "InlineBase64"
            }
        ]
    }
}

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header, data=json.dumps(body))

if response.status_code == 202:
    time.sleep(int(response.headers.get('Retry-After')))
    for retry in range(5):
        responseStatus = requests.request(method='get', url=response.headers.get('Location'), headers=header)
        print(responseStatus.text)
        print(responseStatus.headers)
        if responseStatus.headers.get('Location') is not None:
            # print(responseStatus.text)
            # print(responseStatus.json())
            # print(responseStatus.headers)
            break
        else:
            time.sleep(int(responseStatus.headers.get('Retry-After')))
else:
    print(f"Error - {response.json()}")

### Get the notebook id

In [ ]:
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }
          
# Need to implement for pagination
response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header)

for notebook in response.json().get('value'):
    if notebook.get('displayName') == 'NB_DW_Load_Cost_Analyzer':
        notebookId = notebook.get('id')
        print(f'{notebookId=}')


### Create Warehouse

In [ ]:
import requests, json, time

body = {"displayName": "WH_SampleData"}

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/warehouses', headers=header, data=json.dumps(body))

if response.status_code == 202:
    print(f'Creating Warehouse {body.get("displayName")}')
    time.sleep(int(response.headers.get('Retry-After')))
    for retry in range(5):
        responseStatus = requests.request(method='get', url=response.headers.get('Location'), headers=header)
        print(responseStatus.text)
        print(responseStatus.headers)
        if responseStatus.json().get('status') != 'Succeeded':
            time.sleep(int(responseStatus.headers.get('Retry-After')))
        else:
            if responseStatus.headers.get('Location') is None: # no result was return but operation completed
                break
            else:
                responseResult = requests.request(method='get', url=responseStatus.headers.get('Location'), headers=header)
                print(f"Succeeded {responseResult.json()}")
                break
else:
    print(response.text)

In [ ]:
import requests, json, time

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/warehouses', headers=header)

for warehouse in response.json().get('value'):
    if warehouse.get('displayName') == 'WH_SampleData':
        WH_SampleData = warehouse
        break
print(WH_SampleData)

### Load data to Lakehouse

In [ ]:
import azure.storage.blob
import pandas as pd
from notebookutils import mssparkutils

spark.conf.set('spark.sql.parquet.int96RebaseModeInWrite', 'LEGACY')
spark.conf.set('spark.sql.parquet.datetimeRebaseModeInWrite', 'LEGACY')

account_url = "https://fabrictutorialdata.blob.core.windows.net"
blob_service_client = azure.storage.blob.BlobServiceClient(account_url)
container_client = blob_service_client.get_container_client('sampledata')

tableList = list()
for blob in container_client.walk_blobs(name_starts_with='WideWorldImportersDW/parquet/tables/', delimiter='/'):
    tableName = blob.name.split('/')[-1].split('.parquet')[0]
    tableList.append(tableName)

for table in tableList:
    print(f'Loading Table: {table}')
    if mssparkutils.fs.exists(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}"):
        mssparkutils.fs.rm(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}", recurse=True)

    for blob in container_client.walk_blobs(name_starts_with=f"WideWorldImportersDW/parquet/tables/", delimiter='/'):
        if f'{table}.parquet' == blob.name.split('/')[-1]:
            dfPandas = pd.read_parquet(f'{account_url}/sampledata/{blob.name}', engine='pyarrow', storage_options={'anon': True})

            if not dfPandas.empty:
                dfSpark = spark.createDataFrame(dfPandas)
                dfSpark.write.format('delta').mode('append').option('mergeSchema', "true").save(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}")
            else:
                print('Bad file', blob.name)

### Load data to Warehouse

In [ ]:
from notebookutils import mssparkutils  
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, DoubleType
import pyodbc, struct, itertools, time, datetime, re, uuid, json

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={WH_SampleData.get("properties").get("connectionString")};Database={WH_SampleData.get("displayName")}'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

def get_result_set(cursor):
    if cursor.description:
        resultList = cursor.fetchall()
        resultColumns = columns = [column[0] for column in cursor.description]
    else:
        resultList = []
        resultColumns = []
    return [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList]

with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
    with conn.cursor() as cursor:
        while retry in range(5):
            cursor.execute(f"""SELECT TABLE_NAME FROM LH_SampleData.INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo'""")
            resultSet = get_result_set(cursor)
            if len(resultSet) > 0:
                break
            else:
                time.sleep(10)
        while cursor.nextset():
            queryMessage += ",".join([str(cursor.messages) if cursor.messages else ""])
            resultSet.append(get_result_set(cursor))
        cursor.commit()

    for table in resultSet:
        with conn.cursor() as cursor:
            query = f"""IF OBJECT_ID('dbo.{table.get("TABLE_NAME")}', 'U') IS NOT NULL DROP TABLE dbo.{table.get("TABLE_NAME")}; CREATE TABLE dbo.{table.get("TABLE_NAME")} AS SELECT * FROM LH_SampleData.dbo.{table.get("TABLE_NAME")}"""
            print(query)
            cursor.execute(query)
            cursor.commit()


### Refresh Fabric Capacity Metrics App so that the latest workspace artifacts are in the model

In [ ]:
import requests, json, time

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.get('https://api.fabric.microsoft.com/v1/workspaces', headers=header)

capacityWorkspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == CapacityMetricsWorkspace][0]
print(f'{capacityWorkspaceId = }')

response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets", headers=header)

datasetId = [dataset.get('id') for dataset in response.json().get('value') if dataset.get('name') == CapacityMetricsDataset][0]
print(f'{datasetId = }')

# tableList = [{"table": "Capacities"}
#             ,{"table": "Workspaces"}
#             ,{"table": "TimePoints"}
#             ,{"table": "TimePointBackgroundDetail"}
#             ,{"table": "Items"}
#             ]
body = {"objects": tableList} # Need to ask Pat what tables are import and what are direct query
# response = requests.post(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes", headers=header, data=json.dumps(body))
response = requests.post(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes", headers=header)

refreshId = response.headers.get('RequestId')
print(f'{refreshId = }')

for attempt in range(12): 
    # https://learn.microsoft.com/en-us/power-bi/connect-data/asynchronous-refresh#get-refreshes
    response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes?$top=1", headers=header)
    if response.json().get('value')[0].get('status') != 'Unknown':
        print(f'Refresh Complete')
        break
    else:
        print(f'Refreshing tables ...')
        time.sleep(10)

### Run the notebook

In [ ]:
import requests, json, time

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

body = {
    "executionData": {
        "parameters": {
            "FabricDWWorkspaceName": {"value": workspaceName, "type": "string"}
            ,"FabricDWName": {"value": "WH_SampleData", "type": "string"}
            ,"ConcurrencyNum": {"value": "1", "type": "int"}
            ,"CapacityMetricsWorkspace": {"value": CapacityMetricsWorkspace, "type": "string"}
            ,"CapacityMetricsDataset": {"value": CapacityMetricsDataset, "type": "string"}
        }
    }
}

# print(json.dumps(body, indent=4))

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{notebookId}/jobs/instances?jobType=RunNotebook', headers=header, data=json.dumps(body))

if response.status_code == 202:
    print('Started Execution of Notebook')
    time.sleep(10) # Sleep for 10 seconds to wait for job to get started. TODO Change in the future to a retry
    print(f"{response.headers=}")
    # jobId = response.headers.get('Location')
    while True:
        # print(f"{response.headers.get('Location')=}")
        responseStatus = requests.request(method="get", url=response.headers.get('Location'), headers=header)
        print(f"Status: {responseStatus.json().get('status')} {"|" + responseStatus.json().get('failureReason') if responseStatus.json().get('failureReason') != None else ''}")
        if responseStatus.json().get('status') in ['Succeeded', 'Failed', 'Completed']:
            print(responseStatus.json().get('status'))
            break
        else:
            time.sleep(120) # Wait 2 minutes to check the status of the run
else:
    print('Error trying to execute')

In [ ]:
# import requests

# response = requests.request(method="get", url=f'https://api.fabric.microsoft.com/v1/workspaces/a3b40eca-afde-448e-bc54-af3d318657b0/items/d20ea153-dbf1-4821-8243-e823bf95bdbb/jobs/instances/237f073e-f403-4d12-a2ae-63377fbe0335', headers=header)
# print(response.status_code)
# print(response.json())
